In [1]:
import numpy as np
from itertools import combinations
from itertools import chain

In [58]:
a = [1, 2, 3, 4, 5]
b = [1,4,3,5,2]

signPermutation([x - 1 for x in b])

(1, [[3, 4, 1]])

In [2]:
def signPermutation(p):
 # explicitly using cycle notation of permutation
 # input: p is a list of permutation indices
 # i.e. [x_sorted, p] = sort(x_unsorted)

        n = len(p)
        visited = [False]*n
        sign = 1
        cycle = []
        for k in range(n):
                if (not visited[k]):
                        ct = k
                        L = 0
                        tempcyc = []
                        while (not visited[ct]):
                                L += 1
                                visited[ct] = True
                                ct = p[ct]
                                tempcyc.append(ct)
                        if len(tempcyc) > 1:
                            cycle.append(tempcyc)
                        if L%2 == 0:
                                sign = -1*sign
        return sign, cycle
    
def hextobin(ini_string):
        return bin(int(ini_string,16))[2:]

In [3]:
# Routine that writes the anstisymmetrizer for any combination
# A(pqrst...):

def antisymmetrizer(string_in):
    from itertools import permutations
    
    L = string_in.split('/')

    ct = 1
    P = list(L[0])
    ASYM = ''
    if len(L) == 2:
        # We do not allow for permutations among indices in each group
        # on either side of the slash
        permlistQ = list(permutations(range(len(L[1]))))
    else:
        # find all permutations of the indices in the group
        sign = []
        cycle = []
        permlistP = []
        strcyc = []
        for p in list(permutations(range(len(L[0])))):
            idx = np.argsort(np.asarray(p))
            s,c = signPermutation(idx)
            temp = ''
            for i in range(len(p)):
                temp += P[p[i]]
                temp2 = ''
                for j in range(len(c)):
                    for k in range(len(c[j])-1):   
                        temp2 += '('+P[p[c[j][0]]]+P[p[c[j][len(c[j])-k-1]]]+')'
            sign.append(c)
            cycle.append(c)
            strcyc.append(temp2)
            permlistP.append(temp)
            
            if s > 0:
                if strcyc[-1] == '':
                    ASYM += '+(1)'
                else:
                    ASYM += '+'+strcyc[-1]
            else:
                ASYM += '-'+strcyc[-1]
                
        print('{}'.format(ASYM))
            
            

    return permlistP,sign,cycle,strcyc


In [113]:
class XOperator:
    
    def __init__(self,dag,index,spin):
        
        self.dag = dag # -1 if annihilator, 1 if creator
        self.spin = spin # '1' if alpha, '0' if beta
        self.index = index # string p, q, r, s, i, j, k, l, a, b, c, d, etc.
        if self.index == 'i' or self.index == 'j' or self.index == 'k' or self.index == 'l':
            self.character = 1 # fixed hole
        elif self.index == 'm' or self.index == 'n' or self.index == 'o' or self.index == 'v':
            self.character = 2 # free hole
        elif self.index == 'a' or self.index == 'b' or self.index == 'c' or self.index == 'd':
            self.character = 3 # fixed particle
        elif self.index == 'e' or self.index == 'f' or self.index == 'g' or self.index == 'h':
            self.character = 4 # free particle
        elif self.index == 'p' or self.index == 'q' or self.index == 'r' or self.index == 's':
            self.character = 5 # generic index
            
        if self.character == 1 or self.character == 2:
            self.Yopdag = -1*self.dag
        elif self.character == 3 or self.character ==4:
            self.Yopdag = self.dag
        else:
            self.Yopdag = 2

class Expression:
    
    def __init__(self,expr_label,sign,weight,matrix_element,operator_list): 
        self.sign = sign
        self.weight = weight
        self.matrix_element = matrix_element # contracted & uncontracted indices
        self.operator_list = operator_list # list of FermiOperator objects
        self.expr_label = expr_label
    
    def is_normal_fermi(self):
        FLAG = True
        N = len(self.operator_list)
        for i in range(N-1):
            X1 = self.operator_list[N-1-i]
            X2 = self.operator_list[N-2-i]
            if X1.ph_character == 2 or X2.ph_character == 2:
                return 'Cannot determine Fermi normal ordering with generic indices'
            else:
                Y1 = YOperator(X1)
                Y2 = YOperator(X2)
                if Y1.dag == 1 and Y2.dag == -1:
                    FLAG = False
        return FLAG
    
    def print_expression(self):
        
        printObj = []
        
        if self.sign == 1:
            printObj += ['+']
        if self.sign == -1:
            printObj += ['-']
            
            
        printObj += ['weight = '+str(self.weight)]
        
        printObj += ['matel = '+str(self.expr_label)+'('+str(self.matrix_element)+')']
        
        temp = []
        for i in range(len(self.operator_list)):
            if self.operator_list[i].spin == 1:
                if self.operator_list[i].dag == 1:
                    temp += [self.operator_list[i].index+'+']
                if self.operator_list[i].dag == -1:
                    temp += [self.operator_list[i].index]
            if self.operator_list[i].spin == 0:
                if self.operator_list[i].dag == 1:
                    temp += ['~'+self.operator_list[i].index+'+']
                if self.operator_list[i].dag == -1:
                    temp += ['~'+self.operator_list[i].index]
                
        printObj += temp     
        
        print(printObj)
        
def contraction_check(list_contr):
    # Would be good to make a quick parser method to check whether a list of full contractions
    # is allowable or not
        return bool

def contraction_fermi(X1,X2):
    # will return a delta function ONLY d(pq) we won't contract terms
    # that give 0 anyway! return '0' is a flag that the contraction doesn't
    # work. In practice, we should try to never perform contractions that are 0
    
    if X1.spin == X2.spin:
    
        # h-h case X^dag X
        # (i,m), (i,p), (m,n)
        if X1.character == 1 or X1.character == 2 or X1.character == 5 and \
           X2.character == 1 or X2.character == 2 or X2.character == 5:
            
            if X1.character == 5 and X2.character == 5: # should really never contract p with q
                return '0'
            elif X1.character == 1 and X2.character == 1: # contraction of two fixed indices
                return '0'
            elif X1.dag == 1 and X2.dag == -1:
                return [X1.index, X2.index]
            else:
                return '0'

        # p-p case X X^dag
        # (a,e), (a,p), (e,f)
        elif X1.character == 3 or X1.character == 4 or X1.character == 5 and \
             X2.character == 3 or X2.character == 4 or X2.character == 5:
            
            if X1.character == 5 and X2.character == 5: # should really never contract p with q
                return '0'
            elif X1.character == 3 and X2.character == 3: # contraction of two fixed indices
                return '0'
            elif X1.dag == -1 and X2.dag == 1:
                return [X1.index, X2.index]
            else:
                return '0'

        else:
            return '0'
    else:
        return '0'
        
    

class VEV_Expression:         
     
        def __init__(self, list_of_expressions):
            
            # list of the operator lists of each expression
            self.operator_list = [list_of_expressions[i].operator_list for i in range(len(list_of_expressions))]
            
            # flattened list of the operators of each expression
            self.operator_list_flat = [list_of_expressions[i].operator_list[j] for i in range(len(list_of_expressions)) \
                                      for j in range(len(list_of_expressions[i].operator_list))]
            
            # number of expressions passed in
            self.num_expr = len(list_of_expressions)
            
            self.num_proj_lines = len(list_of_expressions[0].operator_list)
            self.num_tot_lines = len(self.operator_list_flat)
            
            # list containing the association of each operator in operator_list_flat to what number expression passed in
            self.operator_list_exp = []
            for i in range(len(list_of_expressions)):
                for j in range(len(list_of_expressions[i].operator_list)):
                    self.operator_list_exp += [i]
            
            self.operator_expr_idx = []
            for i in range(len(list_of_expressions)):
                self.operator_expr_idx.append([j for j in range(len(self.operator_list_flat)) if \
                                              self.operator_list_exp[j] == i])
            
            # flattened list of indices for each operator
            self.indices = [self.operator_list_flat[i].index for i in range(len(self.operator_list_flat))]
            
            # flattened list of symmetry label (i.e. spin + abelian irrep) for each operator
            self.sym = [self.operator_list_flat[i].spin for i in range(len(self.operator_list_flat))]
            
            # flattened list of character (i.e. fixed/free particle/hole/generic) for each operator
            self.characters = [self.operator_list_flat[i].character for i in range(len(self.operator_list_flat))]
            
            # number of symmetry representations, including spin (= 2*number of abelian irreps for spatial symmetry)
            self.nrep = len(list(set(self.sym)))
            
            # sign, weight, and matrix element
            self.sign = 1
            self.weight = 1
            self.matrix_element = []
            self.expr_labels = []
            for E in list_of_expressions:
                self.sign *= E.sign
                self.weight *= E.weight
                self.matrix_element.append(E.matrix_element)
                self.expr_labels.append(E.expr_label)
                
        def organize_indices(self):
            # generate a list where each element is a list of indices (numerical index) of operators in VEV
            # that have the same spin/symmetry label and hole/particle character (generic indices always included)
            # Returns
            # idx_hole_list:
            # e.g. [[i], [p,r], [m, o]] <- hole_alpha (numerical indices)
            
            idx_hole_list = []
            idx_particle_list = []
            
            for i in range(self.nrep):
                if self.nrep > 1:
                    temp = [j for j in range(len(self.indices)) if i == self.sym[j]]
                else:
                    temp = list(range(len(self.indices)))
                
                tempx = [temp[j] for j in range(len(temp)) if self.characters[temp[j]]==2]
                tempy = [temp[j] for j in range(len(temp)) if self.characters[temp[j]]==4]
                
                temp2 = []
                temp3 = []
                
               # temp2.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 1])
                temp2.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 5])
                temp2.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 2])
                
               # temp3.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 3])
                temp3.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 5])
                temp3.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 4])
                
            # Hole/Particle lines always extend to left on R and T operators, so operators to the right of H_N
            # cannot contract among themselves!
            
#                 temp4 = []
#                 temp5 = []
#                 for k in range(self.num_expr):
#                     temp6 = [tempx[v] for v in range(len(tempx)) if self.operator_list_exp[tempx[v]] == k]
#                     temp7 = [tempy[v] for v in range(len(tempy)) if self.operator_list_exp[tempy[v]] == k]
#                     if len(temp6) > 0:
#                         temp4.append(temp6)
#                     if len(temp7) > 0:
#                         temp5.append(temp7)
                    

#                 idx_hole_list.append(temp2+temp4)
#                 idx_particle_list.append(temp3+temp5)
    
                idx_hole_list.append(temp2)
                idx_particle_list.append(temp3)
    
            return idx_hole_list, idx_particle_list
        
        
        def contraction_check(self,p1,p2):
            
            # connectedness here is defined as contracting a generic index to a 
            # FREE hole/particle index!
            
            X1 = self.operator_list_flat[p1]
            X2 = self.operator_list_flat[p2]
            if X1.character != 5 and X2.character != 5:
                FLAG = True
                connect = False
            else:
                X1 = self.operator_list_flat[p1]
                X2 = self.operator_list_flat[p2]
                if X1.character == 1 and X2.character == 5:
                    connect = False
                    if X1.dag == 1 and X2.dag == -1:
                        FLAG = True
                    else:
                        FLAG = False
                elif X1.character == 5 and X2.character == 2:
                    connect = True
                    if X1.dag == 1 and X2.dag == -1:
                        FLAG = True
                    else:
                        FLAG = False
                elif X1.character == 3 and X2.character == 5:
                    connect = False
                    if X1.dag == -1 and X2.dag == 1:
                        FLAG = True
                    else:
                        FLAG = False
                elif X1.character ==5 and X2.character == 4:
                    connect = True
                    if X1.dag == -1 and X2.dag == 1:
                        FLAG = True
                    else:
                        FLAG = False
                else:
                    FLAG = False
                    
            return FLAG, connect
                         
        
        def generate_pair_list(self):
            
            idx_h_list, idx_p_list = self.organize_indices()
            
            # hole (particle) creators (annhiliators) need generic annhiliators (creators)
            # hole (particle) annihilators (creators) needs generic creators (annhilators)
            
            n_sym_rep = len(idx_h_list) # should equal len(idx_p_list)
            
#             contr_h = []
#             contr_p = []
#             connect_h = []
#             connect_p = []
            
            ct_pair = 0
        
            pair_list = []
            connect_idx = [[] for i in range(self.num_expr-2)]
            
            for n in range(n_sym_rep):
                LH = idx_h_list[n]
                LP = idx_p_list[n]
                
                #temp_h = []
                #temp_p = []
                
                for p in range(len(LH)-1):
                    for q in range(p+1,len(LH)):
                        for i in range(len(LH[p])):
                            for j in range(len(LH[q])):
                                allowed, connect = self.contraction_check(LH[p][i],LH[q][j])
                                if allowed:
                                    pair_list += [(LH[p][i], LH[q][j])]
                                    if connect:
                                        K = max([self.operator_list_exp[LH[p][i]], self.operator_list_exp[LH[q][j]]])
                                        connect_idx[K-2] += [ct_pair]
                                        #connect_idx += [ct_pair]
                                    ct_pair += 1
                                        
                for p in range(len(LP)-1):
                    for q in range(p+1,len(LP)):
                        for i in range(len(LP[p])):
                            for j in range(len(LP[q])):
                                allowed, connect = self.contraction_check(LP[p][i],LP[q][j])
                                if allowed:
                                    pair_list += [(LP[p][i], LP[q][j])]
                                    if connect:
                                        K = max([self.operator_list_exp[LP[p][i]], self.operator_list_exp[LP[q][j]]])
                                        connect_idx[K-2] += [ct_pair]
                                        #connect_idx += [ct_pair]
                                    ct_pair += 1
                                        
                                    
                                    
#                 contr_h.append(temp_h)
#                 contr_p.append(temp_p)

#                 contr_h.append( [(LH[p][i],LH[q][j]) for p in range(len(LH)-1) for q in range(p+1,len(LH)) \
#                                 for i in range(len(LH[p])) for j in range(len(LH[q]))\
#                                   if self.contraction_check(LH[p][i],LH[q][j])] ) 
                
                
#                 LP = idx_p_list[n]
#                 contr_p.append( [(LP[p][i],LP[q][j]) for p in range(len(LP)-1) for q in range(p+1,len(LP)) \
#                                 for i in range(len(LP[p])) for j in range(len(LP[q]))\
#                                 if self.contraction_check(LP[p][i],LP[q][j])] )
                
#             return contr_h, contr_p, connect_h, connect_p
            return pair_list, connect_idx

        
        def generate_depend_matrix(self, pair_list):
            from itertools import chain
            
            # depMat[i] contains indices in pair_list that are not allowed once
            # pair_list[i] is selected as a contraction
            
           # pair_list = self.generate_pair_list()
              
            depMat = []

            for j1 in range(len(pair_list)):
                A = list(chain.from_iterable([pair_list[j1]]))
                temp = []
                for j2 in range(len(pair_list)):
                    B = list(chain.from_iterable([pair_list[j2]]))
                    if any(i in A for i in B):
                        if j1 != j2:
                            temp += [j2]
                depMat.append(temp)
                        
            return depMat
        
        def generate_full_contractions(self):
            from itertools import combinations
            
            FC_list = []
            fc_pairs = []
            
            def common_member(a, b): 
                a_set = set(a) 
                b_set = set(b) 
                if len(a_set.intersection(b_set)) > 0: 
                    return(True)  
                return(False)   
            
            def is_connected(idx, connect_idx):
                if all([common_member(idx, connect_idx[k]) for k in range(self.num_expr-2)]):
                    return True
                else:
                    return False
            
            def is_allowed(idx,depMat):
                from itertools import chain
                b = chain.from_iterable([depMat[i] for i in idx])
                if common_member(idx,b):
                    return(False)
                else:
                    return(True)
            
            pair_list, connect_idx = self.generate_pair_list()
            depMat = self.generate_depend_matrix(pair_list)
            #num_contr = int(len(self.operator_list_flat)/2)
            num_contr = int( (self.num_tot_lines-2*self.num_proj_lines)/2 )
            
            idx = list(combinations(range(len(pair_list)),num_contr))
            
            for i in range(len(idx)):
                print(i)
                if is_allowed(idx[i],depMat) and is_connected(idx[i],connect_idx):
                    FC_list.append(idx[i])
                    fc_pairs.append([pair_list[idx[i][j]] for j in range(len(idx[i]))])
            
            return FC_list, fc_pairs
        
        
        def collect_distinct_diagrams(self, fc_pairs, idx_hole_list, idx_particle_list):

            def common_member(a, b): 
                a_set = set(a) 
                b_set = set(b) 
                if len(a_set.intersection(b_set)) > 0: 
                    return(True)  
                return(False)  

            R_diagrams = []
            CT_H = []
            CT_P = []
            mult = []

            for i in range(len(fc_pairs)):
                ct_h = [0]*len(idx_hole_list)
                ct_p = [0]*len(idx_particle_list)
                IDX = fc_pairs[i][0][1]
                for j in range(len(idx_hole_list)):
                    if IDX in idx_hole_list[j]:
                        ct_h[k] += 1
                    if IDX in idx_hole_list[j]:
                        ct_p[k] += 1
                if i == 0:
                    CT_H.append(ct_h)
                    CT_P.append(ct_p)
                    mult.append(1.0)
                else:
                    flag = False
                    for l in range(len(CT_H)):
                        if all(list(chain.from_iterable(CT_H[l])) == list(chain.from_iterable(ct_h))) and \
                           all(list(chain.from_iterable(CT_P[l])) == list(chain.from_iterable(ct_p))):
                            mult[l] += 1
                            flag = True
                    if not flag:
                        CT_H.append(ct_h)
                        CT_P.append(ct_p)
                        mult.append(1.0)
            return CT_H, CT_P, mult

        
#         def contract_matrix_elements(self,fc_pairs):
#             # return a list iof expression objects
            
#             matel = self.matrix_element.copy()
            
#             def index_replacement(matel, idx):
#                 indice = [self.operator_list_flat[idx[i]].index for i in range(len(idx))]
                
                    
                    
#             for i in range(len(fc_pairs)):
           
        
#         def organize_projection_lines(self,FC_list):
#             for i in range(FC_list):
                
        
        
        def print_full_contractions(self,FC_list,fc_pairs):
        
            for i in range(len(FC_list)):
                temp = []
                for j in range(len(fc_pairs[i])):
                    I = fc_pairs[i][j][0]
                    J = fc_pairs[i][j][1]
                    XI = self.operator_list_flat[I]
                    XJ = self.operator_list_flat[J]
                    strI = XI.index
                    strJ = XJ.index
                    if XI.dag == 1:
                        strI += '+'
                    if XJ.dag == 1:
                        strJ += '+'
                    if XI.spin == 0:
                        strI = '~'+strI
                    if XJ.spin == 0:
                        strJ = '~'+strJ
                    temp += [(strI,strJ)]
                print(temp)
            return
    
        def print_expression(self):
        
            printObj = []

            if self.sign == 1:
                printObj += ['+']
            if self.sign == -1:
                printObj += ['-']


            printObj += ['weight = '+str(self.weight)]
            
            temp3 = ['matel = ']
            for i in range(self.num_expr):
                temp3 += [str(self.expr_labels[i]+'('+self.matrix_element[i]+')')]
                
            printObj += [temp3]

            temp = []
            for i in range(len(self.operator_list)):
                temp2 = []
                for j in range(len(self.operator_list[i])):
                    if self.operator_list[i][j].spin == 1:
                        if self.operator_list[i][j].dag == 1:
                            temp2 += [self.operator_list[i][j].index+'+']
                        if self.operator_list[i][j].dag == -1:
                            temp2 += [self.operator_list[i][j].index]
                    if self.operator_list[i][j].spin == 0:
                        if self.operator_list[i][j].dag == 1:
                            temp2 += ['~'+self.operator_list[i][j].index+'+']
                        if self.operator_list[i][j].dag == -1:
                            temp2 += ['~'+self.operator_list[i][j].index]
                temp.append(temp2)
            printObj.append(temp)    
            
            print(printObj)
            
        
        
#         def generate_full_contractions(self):
            
#             pair_list = self.generate_pair_list()
#             depMat = self.generate_depend_matrix(pair_list)
#             num_contr = int(len(self.operator_list_flat)/2)

#             def remove_indices(my_list,indexes):
#                 for index in sorted(indexes, reverse=True):
#                     del my_list[index]
#                 return my_list
            
#             def recursor(in_pair, pair_list, idx):
                
#                 if len(in_pair) == num_contr:
#                     return in_pair
#                 else:
#                     in_pair += pair_list[0]
#                     pair_list = remove_indices(pair_list, depMat[idx])
#                     recursor(in_par, pair_list, idx+1)
                    
#             for i in range()
                
    
            
#             depMat = [[J for J in range(len(N)) if J in dep_pair_idx] \
#                       for dep_pair_idx in range(len(N)) if \
#                       not any(dep_pair_idx in list(chain.from_iterable([pair_list])))
            
# list_y = list(chain.from_iterable([y]))
#                 list_x = list(chain.from_iterable([x]))
#                 if not any(i in list_x for i in list_y)


#         def organize_indices(self):
            
#             hole_list = []
#             particle_list = []
#             idx_hole_list = []
#             idx_particle_list = []
            
#             # append fixed indices
#             hole_list.append([self.indices[i] for i in range(len(self.characters)) if self.characters[i] == 1])
#             idx_hole_list.append([i for i in range(len(self.characters)) if self.characters[i] == 1])
#             particle_list.append([self.indices[i] for i in range(len(self.characters)) if self.characters[i] == 3])
#             idx_particle_list.append([i for i in range(len(self.characters)) if self.characters[i] == 3])
            
#             # append free indices
#             for i in range(len(self.operator_list)):
#                 temp1 = []
#                 idx_temp1 = []
#                 temp1.append([self.operator_list[i][j].index \
#                                  for j in range(len(self.operator_list[i])) if self.operator_list[i][j].character == 2])
                
#                 temp2 = []
#                 temp2.append([self.operator_list[i][j].index \
#                                  for j in range(len(self.operator_list[i])) if self.operator_list[i][j].character == 4])
                
#                 if len(temp1[0]) > 0:
#                     hole_list += temp1
#                     temp3 = []
#                     for i in range(len(hole_list[-1])):
#                         temp3 += [j for j in range(len(self.indices)) if self.indices[j] == hole_list[-1][i]]
#                     idx_hole_list.append(temp3)
#                 if len(temp2[0]) > 0:
#                     particle_list += temp2
#                     temp4 = []
#                     for i in range(len(particle_list[-1])):
#                         temp4 += [j for j in range(len(self.indices)) if self.indices[j] == particle_list[-1][i]]
#                     idx_particle_list.append(temp4)
            
#             # append generic indices
#             gen_list = [self.indices[i] for i in range(len(self.characters)) if self.characters[i] == 5]
#             idx_gen_list = [i for i in range(len(self.characters)) if self.characters[i] == 5]
#             hole_list.append(gen_list)
#             idx_hole_list.append(idx_gen_list)
#             particle_list.append(gen_list)
#             idx_particle_list.append(idx_gen_list)
            
#             return hole_list, idx_hole_list, particle_list, idx_particle_list
             

In [114]:
class Node:
    import copy 
    
    def __init__(self, expression):
        self.val = expression
        self.left = None
        self.right = None
        
    def insert_left(self, child):
        if self.left is None:
            self.left = child
        else:
            child.left = self.left
            self.left = child
            
    def insert_right(self, child):
        if self.right is None:
            self.right = child
        else:
            child.right = self.right
            self.right = child
        
    def split(self):
        
        def remove_indices(my_list,indexes):
            for index in sorted(indexes, reverse=True):
                del my_list[index]
            return my_list
        
        def swapPositions(my_list, pos): 
            # popping both the elements from list 
            x1 = my_list.pop(pos[0])    
            x2 = my_list.pop(pos[1]-1) 
            # inserting in each others positions 
            my_list.insert(pos[0], x2)   
            my_list.insert(pos[1], x1)   
            return my_list

        isNormal, idx = self.sweepLR()
        
        if isNormal:
            self.left = None
            self.right = None
        else:
            TEMP1 = copy.copy(self.val.operator_list)
            TEMP2 = copy.copy(self.val.operator_list)
            
            # left (delta) node
            matrix_element = 'd('+self.val.operator_list[idx].index + self.val.operator_list[idx+1].index+')' + \
                              self.val.matrix_element
            L_operator_list = remove_indices(TEMP1,[idx,idx+1])
            #L_operator_list = remove_indices(self.val.operator_list,[idx,idx+1])
            child_left = Node(Expression(self.val.expr_label,self.val.sign,self.val.weight,matrix_element,L_operator_list))
            #self.left = Expression(self.val.expr_label,self.val.sign,self.val.weight,matrix_element,L_operator_list)
            self.insert_left(child_left)
            
            # right (N-string) node
            R_operator_list = swapPositions(TEMP2,[idx,idx+1])
            #R_operator_list = swapPositions(self.val.operator_list,[idx,idx+1])
            child_right = Node(Expression(self.val.expr_label,self.val.sign*-1,self.val.weight,self.val.matrix_element,R_operator_list))
            #self.right = Expression(self.val.expr_label,self.val.sign*-1,self.val.weight,self.val.matrix_element,R_operator_list)
            self.insert_right(child_right)
            
    def sweepLR(self):
        flag = True
        idx = -1
        TEMP = copy.deepcopy(self.val.operator_list)
        for i in range(len(self.val.operator_list)-1):
            Y1 = TEMP[i]
            Y2 = TEMP[i+1]
            if Y1.spin != Y2.spin:
                continue
            else:
                if Y1.character == 1 or Y1.character == 2:
                    Y1.dag = -1*Y1.dag
                if Y2.character == 3 or Y2.character == 4:
                    Y2.dag = Y2.dag
                # generic indices not handled yet
                if Y1.dag < Y2.dag:
                    flag = False
                    idx = i
                
        return flag, idx
    
class Tree:
    
    def __init__(self,expr):
        self.nodes = []
        self.nodes.append([Node(expr)])
        
    def recursive_split(self):
        
        stop_flag = [False]
        
        while not all(stop_flag):
            for depth in range(len(self.nodes)):
                print(len(self.nodes))
                TEMP = []
                for J in range(len(self.nodes[depth])):
                    node = self.nodes[depth][J]
                    #if node.left == None and node.right == None:
                    node.split()
                    TEMP += [node.left, node.right]
                    if node.right == None and node.left == None:
                        stop_flag += [True]
                        break
#                 self.nodes.append(TEMP)
                else:
                    self.nodes.append(TEMP)
        
# def normal_order(expression):
#     Tree = Node(expression)
#     Tree.split()
#     while Tree.left == None and Tree.right == None:


    def print_tree(self):
        self.root.print_expression()
        if self.left is not None:
            self.left.print_expression()
        if self.right is not None:
            self.right.print_expression()
        return

In [115]:
X1 = XOperator(1,'p',1)
X2 = XOperator(1,'q',1)
X3 = XOperator(-1,'s',1)
X4 = XOperator(-1,'r',1)

V = Expression('V',1,0.25,'pqrs',[X1,X2,X3,X4])

F = Expression('F',1,1,'pq',[X1,X3])

X5 = XOperator(1,'i',1)
X6 = XOperator(-1,'j',1)
X7 = XOperator(-1,'b',1)
X8 = XOperator(1,'a',1)

E2 = Expression('E',1,1,'ijab',[X5,X6,X7,X8])

X9 = XOperator(1,'e',1)
X10 = XOperator(-1,'m',1)

R1 = Expression('R1',1,1,'em',[X9,X10])

X11 = XOperator(1,'f',1)
X12 = XOperator(1,'g',1)
X13 = XOperator(-1,'o',1)
X14 = XOperator(-1,'n',1)

# what's the weight on the T2 operator?
T2 = Expression('T2',1,0.25,'fgno',[X11,X14,X12,X13])

X15 = XOperator(1,'e',1)
X16 = XOperator(1,'h',1)
X17 = XOperator(-1,'v',1)
X18 = XOperator(-1,'m',1)

T22 = Expression('T2',1,0.25,'ehmv',[X15,X16,X17,X18])

VEV1 = VEV_Expression([E2,V,T2,T22])
VEV1.print_expression()

['+', 'weight = 0.015625', ['matel = ', 'E(ijab)', 'V(pqrs)', 'T2(fgno)', 'T2(ehmv)'], [['i+', 'j', 'b', 'a+'], ['p+', 'q+', 's', 'r'], ['f+', 'n', 'g+', 'o'], ['e+', 'h+', 'v', 'm']]]


In [116]:
E2.print_expression()

['+', 'weight = 1', 'matel = E(ijab)', 'i+', 'j', 'b', 'a+']


In [117]:
N1 = Node(E2)

N1.val.print_expression()

N1.split()
N1.left.val.print_expression()
N1.right.val.print_expression()


N1.left.split()
N1.left.val.print_expression()
N1.right.val.print_expression()

['+', 'weight = 1', 'matel = E(ijab)', 'i+', 'j', 'b', 'a+']


NameError: name 'copy' is not defined

In [89]:
print(N1.left.left)

AttributeError: 'NoneType' object has no attribute 'left'

In [90]:
Tr1 = Tree(E2)
Tr1.recursive_split()

1


NameError: name 'copy' is not defined

In [91]:
for i in range(len(Tr1.nodes)):
    for j in range(len(Tr1.nodes[i])):
        N = Tr1.nodes[i][j]
        if N != None:
            N.val.print_expression()

['+', 'weight = 1', 'matel = E(ijab)', 'i+', 'j', 'b', 'a+']


In [92]:
Tr1.nodes[0][0].val

In [93]:
type(N2)

NameError: name 'N2' is not defined

In [94]:
print_tree(N1)

NameError: name 'print_tree' is not defined

In [118]:
idx_hole_list, idx_particle_list = VEV1.organize_indices()

In [119]:
FC_list, fc_pairs = VEV1.generate_full_contractions()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [120]:
fc_pairs

[[(4, 9), (5, 11), (6, 8), (7, 12)],
 [(4, 9), (5, 11), (6, 8), (7, 13)],
 [(4, 9), (5, 11), (6, 10), (7, 12)],
 [(4, 9), (5, 11), (6, 10), (7, 13)],
 [(4, 9), (5, 11), (6, 12), (7, 8)],
 [(4, 9), (5, 11), (6, 12), (7, 10)],
 [(4, 9), (5, 11), (6, 12), (7, 13)],
 [(4, 9), (5, 11), (6, 13), (7, 8)],
 [(4, 9), (5, 11), (6, 13), (7, 10)],
 [(4, 9), (5, 11), (6, 13), (7, 12)],
 [(4, 9), (5, 14), (6, 8), (7, 10)],
 [(4, 9), (5, 14), (6, 8), (7, 12)],
 [(4, 9), (5, 14), (6, 8), (7, 13)],
 [(4, 9), (5, 14), (6, 10), (7, 8)],
 [(4, 9), (5, 14), (6, 10), (7, 12)],
 [(4, 9), (5, 14), (6, 10), (7, 13)],
 [(4, 9), (5, 14), (6, 12), (7, 8)],
 [(4, 9), (5, 14), (6, 12), (7, 10)],
 [(4, 9), (5, 14), (6, 12), (7, 13)],
 [(4, 9), (5, 14), (6, 13), (7, 8)],
 [(4, 9), (5, 14), (6, 13), (7, 10)],
 [(4, 9), (5, 14), (6, 13), (7, 12)],
 [(4, 9), (5, 15), (6, 8), (7, 10)],
 [(4, 9), (5, 15), (6, 8), (7, 12)],
 [(4, 9), (5, 15), (6, 8), (7, 13)],
 [(4, 9), (5, 15), (6, 10), (7, 8)],
 [(4, 9), (5, 15), (6, 10)

In [112]:
VEV1.print_full_contractions(FC_list,fc_pairs)

[('p+', 'n'), ('q+', 'o'), ('s', 'f+'), ('r', 'e+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'f+'), ('r', 'h+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'g+'), ('r', 'e+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'g+'), ('r', 'h+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'e+'), ('r', 'f+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'e+'), ('r', 'g+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'e+'), ('r', 'h+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'h+'), ('r', 'f+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'h+'), ('r', 'g+')]
[('p+', 'n'), ('q+', 'o'), ('s', 'h+'), ('r', 'e+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'f+'), ('r', 'g+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'f+'), ('r', 'e+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'f+'), ('r', 'h+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'g+'), ('r', 'f+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'g+'), ('r', 'e+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'g+'), ('r', 'h+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'e+'), ('r', 'f+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'e+'), ('r', 'g+')]
[('p+', 'n'), ('q+', 'v'), ('s', 'e+'), ('r', 

In [121]:
CT_H, CT_P, mult = VEV1.collect_distinct_diagrams(fc_pairs, idx_hole_list, idx_particle_list)

TypeError: 'int' object is not iterable

In [25]:
pair_list, connect_idx = VEV1.generate_pair_list()
depMat = VEV1.generate_depend_matrix(pair_list)

In [104]:
fc_pairs[0]

[(4, 7)]

In [23]:
connect_idx

[[0, 3, 6, 9], [1, 2, 4, 5, 7, 8, 10, 11]]

In [26]:
depMat

[[1, 2, 3],
 [0, 2, 4],
 [0, 1, 5],
 [0, 4, 5],
 [1, 3, 5],
 [2, 3, 4],
 [7, 8, 9],
 [6, 8, 10],
 [6, 7, 11],
 [6, 10, 11],
 [7, 9, 11],
 [8, 9, 10]]

In [ ]:
def wick_expand(expression):
    from itertools import combinations
    
    N = len(expression.operator_list)
    
    num_contractions = int(N/2)
    
    WKTM = []
    contr = []
    
    comb_list = generate_contraction_indices(len(expression.operator_list), num_contractions)
    
    # need K nested loops like 
    # for i = 1:m
    #    for j = i+1:m
    #       for k = j+1:m
    #            ....
    # etc to enumerate all K-fold contractions
    
    for k in range(num_contractions):
        idx = comb_list[k]
        if k > 0:
            for i in range(int(len(idx)/2)):
                contr.append([idx[i]])
                NExp = NExpression(expression, [idx[i]])
                NExp.permute_to_N()
                WKTM.append(NExp)
                
        else:
            for i in range(len(idx)):
                contr.append([idx[i]])
                NExp = NExpression(expression, [idx[i]])
                NExp.permute_to_N()
                WKTM.append(NExp)
    
    return WKTM, contr

def generate_contraction_indices(N,num_contractions):

    from itertools import combinations
    from itertools import chain
    
    idx = []
    
    def recursor(ck,c2):
        ckplus1 = []
        for x in ck:
            for y in c2:
                list_y = list(chain.from_iterable([y]))
                list_x = list(chain.from_iterable([x]))
                if not any(i in list_x for i in list_y):
                    temp = x + y
                    ckplus1.append(temp)
                    #ckplus1.append([(temp[i],temp[i+1]) for i in range(len(temp)-1)])
        return ckplus1
    
    X = list(range(N))
    c2 = list(combinations(X, 2))
    
    for i in range(num_contractions):
        print(i)
        if i == 0:
            ck = c2.copy()
        else:
            ck = recursor(ck,c2)
        idx.append(ck)
            
    
    return idx

        
def print_wick_terms(Wick_RHS, all_contr, print_zero):
    if print_zero == 1:
        print('Printing all terms on RHS of Wick Thm...')
        print('\n')
    else:
        print('Printing all non-zero terms on RHS of Wick Thm...')
        print('\n')
    for i in range(len(Wick_RHS)):
        if print_zero == 1:
            print('Contraction: {}'.format(all_contr[i]))
            Wick_RHS[i].print_expression()
        else:
            if Wick_RHS[i].matrix_element != 0.0:
                print('Contraction: {}'.format(all_contr[i]))
                Wick_RHS[i].print_expression()
    return
        

In [ ]:
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    if len(a_set.intersection(b_set)) > 0: 
        return(True)  
    return(False)      

def is_allowed(idx,depMat):
    from itertools import chain
    b = chain.from_iterable([depMat[i] for i in idx])
    if common_member(idx,b):
        return(False)
    else:
        return(True)

In [ ]:
def remove_indices(my_list,indexes):
    for index in sorted(indexes, reverse=True):
        del my_list[index]
    return my_list

In [ ]:
 def combinations_v2(iterable ,r, depMat):
        
    def remove_indices(my_list,indexes):
        for index in sorted(indexes, reverse=True):
            del my_list[index]
        return my_list
        
    def update_pool(pool,indices,depMat):
        

    pool = tuple(iterable)
    
    n = len(pool)
    
    # incompatible number of objects being selected
    if r > n: 
        return
    
    # ordered list 
    indices = list(range(r))
    
    yield tuple(pool[i] for i in indices) # first combination = first r elements
    
    while True:
        
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                print(indices)
                break
        else: # associated with for loop - executes if break is never reached
            return 
        
        indices[i] += 1
        
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
            
        yield tuple(pool[i] for i in indices)

In [ ]:
def pairs(n):
    if (n%2==1 or n<2):
        print("no solution")
        return
    if (n==2):
        yield(  [[0,1]]  )
    else:
        Sn_2 = pairs(n-2) 
        for s in Sn_2:
            yield( s + [[n-2,n-1]] )
            for i in range(int(n/2)-1):
                sn = list(s)
                sn.remove(s[i])
                yield( sn + [ [s[i][0], n-2] , [s[i][1], n-1] ] )
                yield( sn + [ [s[i][1], n-2] , [s[i][0], n-1] ] )

In [10]:
a = [1, 2, 3, 4, 5]

In [12]:
idx = 2
a[:idx]

[1, 2]